##  Поднятие сервера PostgreSQL в Docker

In [4]:
! docker-compose up -d

 Container test_task-db-1  Recreate
 Container test_task-db-1  Recreated
 Container test_task-db-1  Starting
 Container test_task-db-1  Started


In [174]:
import psycopg2

## Получение доступа к PostgreSQL с помощью psycopg2

In [175]:
db_host = 'localhost'
db_port = '5432'
db_name = 'test'
db_user = 'postgres'
db_password = 'postgres'  

def create_connection():
    conn = psycopg2.connect(
        host=db_host,
        port=db_port,
        dbname=db_name,
        user=db_user,
        password=db_password
    )
    return conn


In [179]:
conn = create_connection()

In [180]:
cur = conn.cursor()

## Проверка коннекшн(на тестовой таблице **test**).

In [181]:
query = '''create table if not exists test(
    test_id serial primary key,
    name varchar(20)
    );'''
cur.execute(query)

In [182]:
query = '''select * from test;'''
cur.execute(query)
result = cur.fetchall()
print(result)


[]


In [183]:
query = '''insert into test(name) values ('Brian');'''
cur.execute(query)


In [184]:
query = '''select * from test;'''
cur.execute(query)
result = cur.fetchall()
print(result)

[(1, 'Brian')]


In [185]:
conn.commit()
conn.close()

## Создание связей (relations) в PostgreSQL

book_store:
    
    book_id типа serial - это уникальный идентификатор книги, который автоматически инкрементируется.
    name типа varchar(30) - это название книги, которое может содержать до 30 символов.
    author_id типа int - это внешний ключ, который ссылается на **author_id** в таблице author.
    genre_id типа int - это внешний ключ, который ссылается на **genre_id** в таблице genre.
    amount типа int - это количество книг в наличии.
    price типа decimal(8, 2) - это цена книги, представленная числом с двумя знаками после запятой.

genre:
    
    genre_id типа serial - это уникальный идентификатор жанра, который автоматически инкрементируется.
    name типа varchar(30) - это название жанра, которое может содержать до 30 символов.

author:
    
    author_id типа serial - это уникальный идентификатор автора, который автоматически инкрементируется.
    name типа varchar(30) - это имя автора, которое может содержать до 30 символов.

In [186]:
conn = create_connection()
cur = conn.cursor()

table_name = 'author'
query = f'''create table if not exists {table_name} (
    author_id serial primary key,
    name varchar(30)
)'''

cur.execute(query)
conn.commit()

table_name = 'genre'
query = f'''create table if not exists {table_name} (
    genre_id serial primary key,
    name varchar(30)
)'''

cur.execute(query)
conn.commit()

table_name = 'book_store'
query = f'''create table if not exists {table_name} (
    book_id serial primary key,
    name varchar(30),
    author_id int,
    genre_id int,
    amount int,
    price decimal(8, 2),
    
    foreign key(author_id) references author(author_id) on delete cascade,
    foreign key(genre_id) references genre(genre_id) on delete cascade
)'''

cur.execute(query)
conn.commit()

## Функции для добавления, извлечения, обновления и удаления данных

In [187]:
def insert(table_name, column_names, value_names, conn = conn):
    cur = conn.cursor()
    cur.execute(f"insert into {table_name}({column_names}) values({value_names})")
    conn.commit()

def select(table_name, columns = '*', conn = conn):
    cur = conn.cursor()
    cur.execute(f"select {columns} from {table_name}")
    result = cur.fetchall()
    return result

def delete(table_name, where_clause, conn = conn):
    cur = conn.cursor()
    cur.execute(f"delete from {table_name} where {where_clause}")
    conn.commit()

def update(table_name, set_clause, where_clause, conn = conn):
    cur=conn.cursor()
    cur.execute(f"update {table_name} set {set_clause} where {where_clause}")
    conn.commit()

In [188]:
insert(table_name = 'genre', column_names = 'name', value_names = "'Action'")
insert(table_name = 'genre', column_names = 'name', value_names = "'Comedy'")
insert(table_name = 'genre', column_names = 'name', value_names = "'Adventure'")
print(select('genre'))


[(1, 'Action'), (2, 'Comedy'), (3, 'Adventure')]


In [189]:
insert(table_name = 'author', column_names = 'name', value_names = "'John'")
insert(table_name = 'author', column_names = 'name', value_names = "'Oscar'")
insert(table_name = 'author', column_names = 'name', value_names = "'Michael'")
print(select('author'))

[(1, 'John'), (2, 'Oscar'), (3, 'Michael')]


In [190]:
insert(table_name = 'book_store', column_names = 'name, author_id, genre_id, amount, price', value_names = "'Tree', 1, 2, 5, 8.32")
insert(table_name = 'book_store', column_names = 'name, author_id, genre_id, amount, price', value_names = "'Random Name', 3, 1, 2, 14.53")
print(select('book_store'))

[(1, 'Tree', 1, 2, 5, Decimal('8.32')), (2, 'Random Name', 3, 1, 2, Decimal('14.53'))]


In [191]:
update(table_name = 'genre', set_clause = "name = 'Thriller'", where_clause = "name = 'Adventure'")
print(select('genre'))


[(1, 'Action'), (2, 'Comedy'), (3, 'Thriller')]


In [192]:
delete(table_name = 'book_store', where_clause = "book_id = 1")
print(select('book_store'))

[(2, 'Random Name', 3, 1, 2, Decimal('14.53'))]


In [193]:
conn.close()